In [ ]:
#| default_exp core

# API

> Basic API for Caddy

# Caddy admin

In [ ]:
#| export
import os, subprocess, httpx, json
from fastcore.utils import *
from httpx import HTTPStatusError, get as xget, post as xpost, patch as xpatch, put as xput, delete as xdelete, head as xhead
from typing import Sequence

## Initial functions

In [ ]:
#| export
def get_id(path):
    "Get a ID full URL from a path"
    if path[0 ]!='/': path = '/'+path
    if path[-1]!='/': path = path+'/'
    return f'http://localhost:2019/id{path}'

In [ ]:
host = 'jph.answer.ai'

In [ ]:
get_id('jph.answer.ai')

'http://localhost:2019/id/jph.answer.ai/'

In [ ]:
#| export
def get_path(path):
    "Get a config full URL from a path"
    if path[0 ]!='/': path = '/'+path
    if path[-1]!='/': path = path+'/'
    return f'http://localhost:2019/config{path}'

In [ ]:
get_path('/apps/tls/automation/policies')

'http://localhost:2019/config/apps/tls/automation/policies/'

In [ ]:
#| export
def gid(path='/'):
    "Gets the id at `path`"
    response = xget(get_id(path))
    response.raise_for_status()
    return dict2obj(response.json())

In [ ]:
#| export
def has_id(id):
    "Check if `id` is set up"
    try: gid(id)
    except HTTPStatusError: return False
    return True

In [ ]:
#| export
def gcfg(path='/', method='get'):
    "Gets the config at `path`"
    f = getattr(httpx, method)
    response = f(get_path(path))
    response.raise_for_status()
    return dict2obj(response.json())

In [ ]:
#| export
def has_path(path):
    "Check if `path` is set up"
    try: gcfg(path)
    except HTTPStatusError: return False
    return True

In [ ]:
gcfg()

In [ ]:
#| export
def pid(d, path='/', method='post'):
    "Puts the config `d` into `path`"
    f = getattr(httpx, method)
    response = f(get_id(path), json=obj2dict(d))
    response.raise_for_status()
    return response.text or None

In [ ]:
#| export
def pcfg(d, path='/', method='post'):
    "Puts the config `d` into `path`"
    f = getattr(httpx, method)
    response = f(get_path(path), json=obj2dict(d))
    try: response.raise_for_status()
    except Exception as e:
        e.add_note(f"Error: '{json.loads(response.text)['error']}'")
        raise
    return response.text or None

In [ ]:
# pcfg({})

In [ ]:
#| export
def nested_setdict(sd, value, *keys):
    "Returns `sd` updated to set `value` at the path `keys`"
    d = sd
    for key in keys[:-1]: d = d.setdefault(key, {})
    d[keys[-1]] = value
    return sd

In [ ]:
nested_setdict({'a':'b'}, {'c':'d'}, 'apps', 'http', 'servers', 'srv0')

{'a': 'b', 'apps': {'http': {'servers': {'srv0': {'c': 'd'}}}}}

In [ ]:
#| export
def path2keys(path):
    "Split `path` by '/' into a list"
    return path.strip('/').split('/')

In [ ]:
path2keys('/apps/tls/automation/policies')

['apps', 'tls', 'automation', 'policies']

In [ ]:
#| export
def keys2path(*keys):
    "Join `keys` into a '/' separated path"
    return '/'+'/'.join(keys)

In [ ]:
keys2path('apps', 'tls', 'automation', 'policies')

'/apps/tls/automation/policies'

In [ ]:
#| export
def nested_setcfg(value, *keys):
    d = nested_setdict(gcfg(), value, *keys)
    return pcfg(d)

In [ ]:
#| export
def init_path(path, skip=0):
    sp = []
    for i,p in enumerate(path2keys(path)):
        sp.append(p)
        if i<skip: continue
        pcfg({}, keys2path(*sp))

## Automation setup

In [ ]:
cf_token = os.environ.get('CADDY_CF_TOKEN', 'XXX')

In [ ]:
#| export
automation_path = '/apps/tls/automation'
def get_acme_config(token):
    prov = { "provider": { "name": "cloudflare", "api_token": token } }
    return { "module": "acme", "challenges": { "dns": prov } }

In [ ]:
#| export
def add_tls_internal_config():
    if has_path(automation_path): return    
    pcfg({})
    init_path(automation_path)
    val = [{"issuers": [{"module": "internal"}]}]
    pcfg(val, automation_path+'/policies')

In [ ]:
#| export
def add_acme_config(cf_token):
    if has_path(automation_path): return
    pcfg({})
    init_path(automation_path)
    val = [get_acme_config(cf_token)]
    pcfg([{'issuers':val}], acme_path+'/policies')

In [ ]:
# add_acme_config(cf_token)

In [ ]:
# gcfg('/apps/tls/automation/policies')[0]

## Route setup

In [ ]:
#| export
srvs_path = '/apps/http/servers'
rts_path = srvs_path+'/srv0/routes'

In [ ]:
#| export
def init_routes(srv_name='srv0', skip=1):
    "Create basic http server/routes config"
    if has_path(srvs_path): return
    init_path(srvs_path, skip=skip)
    ir = {'listen': [':80', ':443'], 'routes': [], 'protocols': ['h1', 'h2']}
    pcfg(ir, f"{srvs_path}/{srv_name}")

In [ ]:
init_routes()

In [ ]:
#| export
def setup_pki_trust(install_trust):
    "Configure PKI certificate authority trust installation"
    if install_trust is None: return
    pki_path = '/apps/pki/certificate_authorities/local'
    init_path(pki_path, skip=1)
    pcfg({"install_trust": install_trust}, pki_path)

In [ ]:
#| export
def setup_caddy(
        cf_token=None, # Cloudflare API token
        srv_name='srv0', # Server name in the Caddyfile
        local:bool=False, # Whether or not this is for localdev or deployment
        install_trust:bool=None): # Install trust store?
    "Create SSL config and HTTP app skeleton"
    if local: add_tls_internal_config()
    else: add_acme_config(cf_token)
    setup_pki_trust(install_trust)
    init_routes(srv_name)

In [ ]:
# pcfg({})
setup_caddy(cf_token)

In [ ]:
# gcfg(srvs_path)

In [ ]:
#| export
def add_route(route):
    "Add `route` dict to config"
    return pcfg(route, rts_path)

In [ ]:
#| export
def del_id(id):
    "Delete route for `id` (e.g. a host)"
    xdelete(get_id(id))

In [ ]:
# del_id(host)

In [ ]:
#| export
def add_reverse_proxy(from_host, to_url):
    "Create a reverse proxy handler"
    if has_id(from_host): del_id(from_host)
    route = {
        "handle": [{ "handler": "reverse_proxy",
                    "upstreams": [{"dial": to_url}] }],
        "match": [{"host": [from_host]}],
        "@id": from_host,
        "terminal": True
    }
    add_route(route)

In [ ]:
host = 'foo.fast.ai'

In [ ]:
# add_reverse_proxy(host, "localhost:5001")

In [ ]:
# gid(host)

In [ ]:
#| export
def add_wildcard_route(domain):
    "Add a wildcard subdomain"
    route = {
        "match": [{"host": [f"*.{domain}"]}],
        "handle": [
            { "handler": "subroute", "routes": [] }
        ],
        "@id": f"wildcard-{domain}",
        "terminal": True
    }
    add_route(route)

In [ ]:
add_wildcard_route('something.fast.ai')

In [ ]:
#| export
def add_sub_reverse_proxy(
        domain,
        subdomain,
        port:str|int|Sequence, # A single port or list of ports
        host='localhost'
    ):
    "Add a reverse proxy to a wildcard subdomain supporting multiple ports"
    wildcard_id = f"wildcard-{domain}"
    route_id = f"{subdomain}.{domain}"
    if isinstance(port, (int,str)): port = [port]
    upstreams = [{"dial": f"{host}:{p}"} for p in port]
    new_route = {
        "@id": route_id,
        "match": [{"host": [route_id]}],
        "handle": [{ "handler": "reverse_proxy", "upstreams": upstreams }]
    }
    pid([new_route], f"{wildcard_id}/handle/0/routes/...")

In [ ]:
add_sub_reverse_proxy('something.fast.ai', 'foo', 5001)

In [ ]:
del_id('foo.something.fast.ai')

## Export -

In [ ]:
#|hide
#|eval: false
from nbdev.doclinks import nbdev_export
nbdev_export()